In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import re
import urllib.request

In [4]:
df = pd.read_csv(r'C:\Users\alber\OneDrive\Documentos\GitHub\DS_ONLINE_THEBRIDGE_ATC\Masterclass\Kaggle\data\train.csv', index_col="laptop_ID")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           912 non-null    object 
 1   Product           912 non-null    object 
 2   TypeName          912 non-null    object 
 3   Inches            912 non-null    float64
 4   ScreenResolution  912 non-null    object 
 5   Cpu               912 non-null    object 
 6   Ram               912 non-null    int32  
 7   Memory            912 non-null    object 
 8   Gpu               912 non-null    object 
 9   OpSys             912 non-null    object 
 10  Weight            912 non-null    float64
 11  Price_in_euros    912 non-null    float64
 12  Cpu_Brand         912 non-null    object 
 13  Cpu_Freq_GHz      853 non-null    float64
 14  Resolution        912 non-null    int64  
 15  Touchscreen       912 non-null    int64  
 16  SSD               912 non-null    int64  
 17  

In [10]:
df.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros,Cpu_Brand,Cpu_Freq_GHz,Resolution,Touchscreen,SSD,HDD,Gpu_Brand,Windows
laptop_ID,,,,,,,,,,,,,,,,,,,,
755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8,256GB SSD,Intel HD Graphics 520,Windows 10,1.86,539.00,Core i3,NaN,2073600,0,0,0,Intel,1
618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59,879.01,Core i7,2.6,2073600,0,0,0,Nvidia,1
909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04,900.00,Core i7,2.7,2073600,0,0,0,Nvidia,1
2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94,Core i5,1.8,1296000,0,0,0,Intel,0
286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4,1TB HDD,AMD Radeon R5 M430,Linux,2.25,428.00,Core i3,2.0,2073600,0,0,0,AMD,0


In [11]:
df.tail()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros,Cpu_Brand,Cpu_Freq_GHz,Resolution,Touchscreen,SSD,HDD,Gpu_Brand,Windows
laptop_ID,,,,,,,,,,,,,,,,,,,,
28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8,256GB SSD,AMD Radeon 530,Windows 10,2.20,800.00,Core i5,1.6,2073600,0,0,0,AMD,1
1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8,256GB SSD,Intel HD Graphics 520,Windows 10,1.48,1629.00,Core i5,2.4,2073600,1,0,0,Intel,1
78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,2TB HDD,Intel HD Graphics 620,No OS,2.20,519.00,Core i5,2.5,2073600,0,0,0,Intel,0
23,HP,255 G6,Notebook,15.6,1366x768,AMD E-Series E2-9000e 1.5GHz,4,500GB HDD,AMD Radeon R2,No OS,1.86,258.00,AMD,1.5,1049088,0,0,0,AMD,0
229,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,4.42,2456.34,Core i7,2.8,2073600,0,0,0,Nvidia,1


In [12]:
df.describe()

,Inches,Ram,Weight,Price_in_euros,Cpu_Freq_GHz,Resolution,Touchscreen,SSD,HDD,Windows
count,912.000000,912.000000,912.000000,912.000000,853.000000,9.120000e+02,912.000000,912.0,912.0,912.000000
mean,14.981579,8.263158,2.026937,1111.724090,2.292122,2.139313e+06,0.143640,0.0,0.0,0.851974
std,1.436719,5.044788,0.665466,687.959172,0.522687,1.347720e+06,0.350917,0.0,0.0,0.355321
min,10.100000,2.000000,0.690000,174.000000,0.900000,1.049088e+06,0.000000,0.0,0.0,0.000000
25%,14.000000,4.000000,1.490000,589.000000,1.800000,1.234272e+06,0.000000,0.0,0.0,1.000000
50%,15.600000,8.000000,2.040000,978.000000,2.500000,2.073600e+06,0.000000,0.0,0.0,1.000000
75%,15.600000,8.000000,2.300000,1483.942500,2.700000,2.073600e+06,0.000000,0.0,0.0,1.000000
max,18.400000,64.000000,4.700000,6099.000000,3.600000,8.294400e+06,1.000000,0.0,0.0,1.000000


In [5]:
# Procesamiento de datos
df["Ram"] = df["Ram"].str.replace("GB", "").astype(int)
df["Weight"] = df["Weight"].str.replace("kg", "").astype(float)

# Extraer información más detallada de la CPU
def extract_cpu_info(cpu):
    parts = str(cpu).split()
    if len(parts) > 2 and parts[0] == "Intel":
        return f"{parts[1]} {parts[2]}"
    return parts[0]
df["Cpu_Brand"] = df["Cpu"].apply(extract_cpu_info)

# Extraer la frecuencia del CPU con manejo de errores
def extract_cpu_freq(cpu):
    if isinstance(cpu, str):
        match = re.search(r'(\d+\.\d+)', cpu)
        return float(match.group(1)) if match else np.nan
    return np.nan
df["Cpu_Freq_GHz"] = df["Cpu"].apply(extract_cpu_freq)

# Extraer resolución de pantalla
def extract_resolution(screen_res):
    match = re.search(r'(\d+)x(\d+)', str(screen_res))
    if match:
        return int(match.group(1)) * int(match.group(2))
    return np.nan
df["Resolution"] = df["ScreenResolution"].apply(extract_resolution)

# Identificar Touchscreen
df["Touchscreen"] = df["ScreenResolution"].apply(lambda x: 1 if "Touchscreen" in str(x) else 0)

# Separar memoria en SSD y HDD
def process_memory(mem):
    ssd, hdd = 0, 0
    parts = str(mem).split()
    for i, part in enumerate(parts):
        if "SSD" in part:
            ssd = int(parts[i-1]) if i > 0 and parts[i-1].isdigit() else 0
        elif "HDD" in part:
            hdd = int(parts[i-1]) if i > 0 and parts[i-1].isdigit() else 0
    return pd.Series([ssd, hdd])
df[['SSD', 'HDD']] = df['Memory'].apply(process_memory)

# Extraer GPU Brand
def extract_gpu_brand(gpu):
    brands = ["Nvidia", "AMD", "Intel"]
    for brand in brands:
        if brand in str(gpu):
            return brand
    return "Other"
df["Gpu_Brand"] = df["Gpu"].apply(extract_gpu_brand)

# Codificar sistema operativo (Windows o no)
df['Windows'] = df['OpSys'].apply(lambda x: 1 if 'Windows' in str(x) else 0)

# Seleccionar características
X = df.drop(columns=["Price_in_euros", "Product", "Cpu", "OpSys", "Memory", "ScreenResolution", "Gpu"])
y = df["Price_in_euros"]

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar variables categóricas y numéricas
categorical_features = ["Company", "TypeName", "Cpu_Brand", "Gpu_Brand"]
numeric_features = ["Inches", "Ram", "Weight", "Resolution", "SSD", "HDD", "Windows", "Cpu_Freq_GHz", "Touchscreen"]

# Preprocesador con imputación para valores NaN
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

# Modelo con optimización de hiperparámetros
param_dist = {
    "regressor__n_estimators": [100, 200, 300],
    "regressor__learning_rate": [0.01, 0.05, 0.1],
    "regressor__max_depth": [3, 5, 7],
    "regressor__min_samples_split": [2, 4, 6]
}

model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", GradientBoostingRegressor(random_state=42))
])

search = RandomizedSearchCV(model, param_dist, cv=3, scoring='neg_root_mean_squared_error', n_iter=10, n_jobs=-1, random_state=42)
search.fit(X_train, y_train)

# Evaluar modelo
best_model = search.best_estimator_
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Optimized RMSE: {rmse}")

Optimized RMSE: 402.42549824567493


In [6]:
# Cargar datos de prueba
X_pred = pd.read_csv(r'C:\Users\alber\OneDrive\Documentos\GitHub\DS_ONLINE_THEBRIDGE_ATC\Masterclass\Kaggle\data\test.csv', index_col="laptop_ID")

# Aplicar el mismo procesamiento
X_pred["Ram"] = X_pred["Ram"].str.replace("GB", "").astype(int)
X_pred["Weight"] = X_pred["Weight"].str.replace("kg", "").astype(float)
X_pred["Cpu_Brand"] = X_pred["Cpu"].apply(extract_cpu_info)
X_pred["Cpu_Freq_GHz"] = X_pred["Cpu"].apply(extract_cpu_freq)
X_pred["Resolution"] = X_pred["ScreenResolution"].apply(extract_resolution)
X_pred["Touchscreen"] = X_pred["ScreenResolution"].apply(lambda x: 1 if "Touchscreen" in str(x) else 0)
X_pred[['SSD', 'HDD']] = X_pred['Memory'].apply(process_memory)
X_pred["Gpu_Brand"] = X_pred["Gpu"].apply(extract_gpu_brand)
X_pred['Windows'] = X_pred['OpSys'].apply(lambda x: 1 if 'Windows' in str(x) else 0)

# Eliminar columnas innecesarias
X_pred = X_pred.drop(columns=["Product", "Cpu", "OpSys", "Memory", "ScreenResolution", "Gpu"])

In [7]:
X_pred

,Company,TypeName,Inches,Ram,Weight,Cpu_Brand,Cpu_Freq_GHz,Resolution,Touchscreen,SSD,HDD,Gpu_Brand,Windows
laptop_ID,,,,,,,,,,,,,
209,Lenovo,Gaming,15.6,16,2.400,Core i7,2.8,2073600,0,0,0,Nvidia,0
1281,Acer,Notebook,15.6,4,2.400,Celeron Dual,1.6,1049088,0,0,0,Intel,0
1168,Lenovo,Notebook,15.6,4,1.900,Core i3,2.0,1049088,0,0,0,Intel,0
1231,Dell,2 in 1 Convertible,15.6,8,2.191,Core i5,2.5,2073600,1,0,0,Intel,1
1020,HP,Notebook,14.0,4,1.950,Core i5,2.5,2073600,0,0,0,Intel,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,MSI,Gaming,17.3,16,2.900,Core i7,2.8,2073600,0,0,0,Nvidia,1
948,Toshiba,Notebook,14.0,4,1.470,Core i5,2.3,2073600,0,0,0,Intel,1
483,Dell,Workstation,15.6,8,1.780,Core i7,2.8,2073600,0,0,0,Nvidia,1


In [8]:
# Hacer predicciones
predictions = best_model.predict(X_pred)


In [13]:
predictions

array([1264.83368628,  301.79422504,  407.138821  ,  830.16724665,
       1050.70017357,  525.64215666,  831.29461331,  939.6511628 ,
       1234.07931316,  372.86163458, 2562.15347455, 1384.50001108,
        549.55961567, 1398.4129208 ,  904.09557412,  743.83334147,
       1987.36224929, 1405.55262954, 1787.22612938,  672.30246821,
       1256.02280536,  316.53353519,  767.32445555,  975.75948113,
        387.51385698,  854.24138556,  628.49263534,  993.08376262,
       2362.54860158, 1100.47936813, 2004.22033132,  463.9625655 ,
        880.97395613, 2980.75782063, 1833.78273264, 1969.278735  ,
        695.01446932, 1556.03338205,  930.50634713, 1535.10678806,
        594.54816495,  757.02948311,  551.68176273, 1133.0099722 ,
       1357.91685457,  992.85987811,  965.89069243,  642.17113006,
        643.72557948,  366.38407961, 1918.73146705,  781.95149284,
       1139.34926464,  423.87489517, 1994.52285585, 2144.01458708,
        616.20647729,  973.06503125, 1007.80305336,  556.97633

In [14]:
sample= pd.read_csv(r'C:\Users\alber\OneDrive\Documentos\GitHub\DS_ONLINE_THEBRIDGE_ATC\Masterclass\Kaggle\data\sample_submission.csv')

In [17]:
submission = pd.DataFrame({"laptop_ID": X_pred.index, "Price_in_euros": predictions})

In [18]:
submission.head()

,laptop_ID,Price_in_euros
0,209,1264.833686
1,1281,301.794225
2,1168,407.138821
3,1231,830.167247
4,1020,1050.700174


In [20]:
submission.shape

(391, 2)

In [21]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission6.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [22]:
chequeador(submission)

You're ready to submit!


NameError: name 'Image' is not defined